# Как рассчитать IP сети на PowerShell

> Этот ноутбук работает на Linux

**Сценарий**

Предположим, что для всего, что мы планируем запускать в Azure, network team выделила нам диапазон адресов `10.172.0.0/16`. При этом мы пытаемся сделать автоматизированный механизм, который, получив на вход желаемый размер или список размеров сетей, сможет найти свободные блоки из указанного диапазона и рассчитать подсети и маски. Затем эти подсети и маски мы сможем использовать для создания сетей в Azure.

Эти примеры можно затем использовать при автоматизации создания того, что Microsoft называет [Landing Zone](https://docs.microsoft.com/en-us/azure/cloud-adoption-framework/ready/landing-zone/), в части автоматического рассчета диапазонов сетей

Начнем c загрузки модуля [ipmgmt](https://github.com/eosfor/ipmgmt)

In [ ]:
Import-Module ipmgmt

Если он не установлен, его всегда можно установить из Powershell gallery

In [ ]:
Install-Module ipmgmt

В этом модуле всего две команды. Одна из них предназначена для разбиения сетей на подсети. Вторая - может найти свободные диапазоны подходящего размера, исходя из "корневой" сети и списка занятых сетей в этой корневой сети

In [ ]:
Get-Command -Module ipmgmt


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-IPRanges                                       0.1.5      ipmgmt
Function        Get-VLSMBreakdown                                  0.1.5      ipmgmt



Теперь разобьем нашу корневую сеть на несколько подсетей. Для нас эти подсети станут виртуальными сетями в Azure. Для этого нам нужно указать эти сети и их размеры. В переменной `$subnets` содержится массив этих будущих сетей. В поле `type`  мы указываем имя будущей сети, а в поле `size` - ее размер. Размер - это количество возможных адресов в сети. В нашем примере мы создаем две /24 сети и их размер соответственно 256 адресов максимально возможных  минус 2 – нулевой и последний которые мы хотим исключить. Не забываем, что Azure отхватывает еще по 5 адресов в каждой подсети, которую мы создаем для своих нужд

In [ ]:
$subnets = @{type = "VNET-HUB"; size = (256-2)},
           @{type = "VNET-A"; size = (256-2)}

Сети готовы, можно разбивать

In [ ]:
Get-VLSMBreakdown -Network 10.172.0.0/16 -SubnetSize $subnets | ft type, network, netmask, *usable, cidr -AutoSize


type     Network      Netmask       FirstUsable  LastUsable     Usable Cidr
----     -------      -------       -----------  ----------     ------ ----
VNET-A   10.172.1.0   255.255.255.0 10.172.1.1   10.172.1.254      254   24
VNET-HUB 10.172.0.0   255.255.255.0 10.172.0.1   10.172.0.254      254   24
reserved 10.172.128.0 255.255.128.0 10.172.128.1 10.172.255.254  32766   17
reserved 10.172.64.0  255.255.192.0 10.172.64.1  10.172.127.254  16382   18
reserved 10.172.32.0  255.255.224.0 10.172.32.1  10.172.63.254    8190   19
reserved 10.172.16.0  255.255.240.0 10.172.16.1  10.172.31.254    4094   20
reserved 10.172.8.0   255.255.248.0 10.172.8.1   10.172.15.254    2046   21
reserved 10.172.4.0   255.255.252.0 10.172.4.1   10.172.7.254     1022   22
reserved 10.172.2.0   255.255.254.0 10.172.2.1   10.172.3.254      510   23



Как видно из результата, мы получили две сети с указанными именами `VNET-A` и `VNET-HUB`. При этом, в корневом диапазоне образовались незанятые слоты, которые получились от того, что сети, которые мы выделяли, маленькие. Однако команда старается максимизировать размеры этих неиспользованных кусков, с тем чтобы терять меньше адресов при таких разбиениях. Так, например, есть кусок с размером `/17` в поле `Cidr`. Это говорит о том, что команда разбила наш `/16` диапазон на два `/17`, оставила один из них неиспользованным, затем разбила другой, уже на два `/18` и так далее. Незадействованные куски считаются "зарезервированными" для будущего использования

При условии, что вы уже авторизовались в Azure. В данном примере я использую ресурсную группу с именем `vnet-test` в регионе `eastus2`

In [ ]:
Login-AzAccount

Сеточки эти теперь можно создать примерно вот так. Тут мы отфильтровываем "reserved" диапазоны - не будем создавать сети из них. Затем, для каждого из этих диапазонов создаем сеточку в некой ресурсной группе. Это, конечно, упрощенный пример, но дальнейшие усовершенствования, связанные с другими подписками или ресурсными группами, пирингом виртуальных сетей и их дальнейшим разбиением на подсети уже никак не повлияет на то, что бы делаем сейчас.

In [ ]:
$vnets = Get-VLSMBreakdown -Network 10.172.0.0/16 -SubnetSize $subnets | ? type -ne 'reserved'

$vnets | % {
    New-AzVirtualNetwork -Name  $_.type -ResourceGroupName 'vnet-test' `
                         -Location 'eastus2' -AddressPrefix "$($_.Network)/$($_.cidr)" | select name, AddressSpace, ResourceGroupName, Location
}

Теперь мы готовы добавлять новые диапазоны к существующим сетям. При этом мы хотим максимально использовать пустые места, чтобы не терять адресов. Для этого в модуле есть команда `Get-IPRanges`. Она принимает на вход список занятых диапазонов, "корневой" диапазон и размер сети в VLSM нотации и возвращает сети, которые нашла. При этом она пытается использовать сначала незанятые диапазоны и предлагает слот в конце диапазона, если он есть.

In [ ]:
Get-IPRanges -Networks "10.10.5.0/24", "10.10.7.0/24" -CIDR 22 -BaseNet "10.10.0.0/16" | ft -AutoSize


IsFree Network   AddressFamily Netmask       Broadcast    FirstUsable LastUsable   Usable Total Cid
                                                                                                  r
------ -------   ------------- -------       ---------    ----------- ----------   ------ ----- ---
  True 10.10.0.0  InterNetwork 255.255.252.0 10.10.3.255  10.10.0.1   10.10.3.254    1022  1024  22
 False 10.10.5.0  InterNetwork 255.255.255.0 10.10.5.255  10.10.5.1   10.10.5.254     254   256  24
 False 10.10.7.0  InterNetwork 255.255.255.0 10.10.7.255  10.10.7.1   10.10.7.254     254   256  24
  True 10.10.8.0  InterNetwork 255.255.252.0 10.10.11.255 10.10.8.1   10.10.11.254   1022  1024  22



В этом примере "базовая" сеть `10.10.0.0/16` два занятых диапазона - `10.10.5.0/24`, `10.10.7.0/24` и длина маски `/22`. Команда нашла два свободных слота длиной `/22` - один до занятых сеток, и один после.

Теперь провернем более хитрый трюк. Что если необходимо найти не один, а сразу много диапазонов с заранее известными размерами. Пи этом, как источник уже существующих сетей - занятых, будем использовать сам Azure, ведь он уже хранит всю необходимую нам информацию.

Сделаем сдедующее: 

- Создадим список желаемых размеров сеток - `$cidrRange`. Сортируем по-возрастанию, чтобы сначала использовать бОльшие диапазоны, если они есть.
- Из Azure подтянем список сеток, которые там есть, тех, которые мы считаем занятыми - `$existingRanges`. 
- Для корректного сравнения нам надо привести эти ranges к типу `System.Net.IPNetwork`, который используется внутри модуля, для рассчетов сетей. 
- Теперь нам надо просто бежать по списку желаемых рамеров сеток, просить `Get-IPRanges` найти слоты и аккумулировать их в сиске используемых - строки 10-13

После всего нам остается лишь пометить найденные диапазоны, как свободные,  сравних их со списком уже занятых, которые мы подтянули из Azure - строки 2-5

In [ ]:
$cidrRange = 25,25,24,24,24,24,23,25,26,26 | sort
$existingRanges = (Get-AzVirtualNetwork -ResourceGroupName vnet-test | 
    select name, @{l = "AddressSpace"; e = { $_.AddressSpace.AddressPrefixes }}, ResourceGroupName, Location |
    select -expand AddressSpace)
$existingNetworks = $existingRanges | % {[System.Net.IPNetwork]$_}
$nets = $existingRanges

$ret = @()

$cidrRange | % {
    $ret = Get-IPRanges -Networks $nets -CIDR $_ -BaseNet "10.172.0.0/16"
    $nets = ($ret | select @{l="range"; e = {"$($_.network)/$($_.cidr)"}}).range
}

$ret | % {
    if ( -not ($_ -in $existingNetworks)) {$_.IsFree = $true}
}

$ret | ft -AutoSize


IsFree Network      AddressFamily Netmask         Broadcast    FirstUsable  LastUsable   Usable Tot
                                                                                                 al
------ -------      ------------- -------         ---------    -----------  ----------   ------ ---
 False 10.172.0.0    InterNetwork 255.255.255.0   10.172.0.255 10.172.0.1   10.172.0.254    254 256
 False 10.172.1.0    InterNetwork 255.255.255.0   10.172.1.255 10.172.1.1   10.172.1.254    254 256
  True 10.172.2.0    InterNetwork 255.255.254.0   10.172.3.255 10.172.2.1   10.172.3.254    510 512
  True 10.172.4.0    InterNetwork 255.255.255.0   10.172.4.255 10.172.4.1   10.172.4.254    254 256
  True 10.172.5.0    InterNetwork 255.255.255.0   10.172.5.255 10.172.5.1   10.172.5.254    254 256
  True 10.172.6.0    InterNetwork 255.255.255.0   10.172.6.255 10.172.6.1   10.172.6.254    254 256
  True 10.172.7.0    InterNetwork 255.255.255.0   10.172.7.255 10.172.7.1   10.172.7.254    254 256

In [ ]:
$s = @()
$ret | % {
    $s += @{type = "VNET-$(Get-Random -Maximum 1000 -Minimum 10)"; size = ($_.total - 2)}
}


$view = Get-VLSMBreakdown -Network 10.172.0.0/16 -SubnetSize $s | sort cidr | ConvertTo-Html -Fragment
[Microsoft.DotNet.Interactive.Kernel]::HTML($view) | Out-Display

type,Network,AddressFamily,Netmask,Broadcast,FirstUsable,LastUsable,Usable,Total,Cidr
reserved,10.172.128.0,InterNetwork,255.255.128.0,10.172.255.255,10.172.128.1,10.172.255.254,32766,32768,17
reserved,10.172.64.0,InterNetwork,255.255.192.0,10.172.127.255,10.172.64.1,10.172.127.254,16382,16384,18
reserved,10.172.32.0,InterNetwork,255.255.224.0,10.172.63.255,10.172.32.1,10.172.63.254,8190,8192,19
reserved,10.172.24.0,InterNetwork,255.255.248.0,10.172.31.255,10.172.24.1,10.172.31.254,2046,2048,21
VNET-795,10.172.0.0,InterNetwork,255.255.254.0,10.172.1.255,10.172.0.1,10.172.1.254,510,512,23
reserved,10.172.23.0,InterNetwork,255.255.255.0,10.172.23.255,10.172.23.1,10.172.23.254,254,256,24
reserved,10.172.21.0,InterNetwork,255.255.255.0,10.172.21.255,10.172.21.1,10.172.21.254,254,256,24
reserved,10.172.19.0,InterNetwork,255.255.255.0,10.172.19.255,10.172.19.1,10.172.19.254,254,256,24
reserved,10.172.17.0,InterNetwork,255.255.255.0,10.172.17.255,10.172.17.1,10.172.17.254,254,256,24
reserved,10.172.15.0,InterNetwork,255.255.255.0,10.172.15.255,10.172.15.1,10.172.15.254,254,256,24
